# Arrival Count Modeling: From Poisson to Negative Binomial

In [9]:
from __future__ import annotations

import sys
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import norm

BASE_DIR = Path.cwd()
for candidate in [BASE_DIR, *BASE_DIR.parents]:
    if (candidate / "src").exists():
        BASE_DIR = candidate
        break
else:
    raise FileNotFoundError("Could not locate project root")

if str(BASE_DIR / "src") not in sys.path:
    sys.path.append(str(BASE_DIR / "src"))

DATA_DIR = BASE_DIR / "data" / "raw"
lookup_path = DATA_DIR / "taxi_zone_lookup.csv"

In [10]:
from modeling.poisson_zone import load_taxi_pickups, attach_zone_metadata, bucket_counts_by_group, poisson_summary
from modeling.nhpp import build_nhpp, cumulative_counts
from modeling.glm_counts import fit_glm, compare_models

In [11]:
yellow_path = DATA_DIR / "yellow_tripdata_2024-01.parquet"
poisson_sample = load_taxi_pickups(yellow_path)
poisson_sample = attach_zone_metadata(poisson_sample, lookup_path)
in_scope = (poisson_sample.dropna(subset=["Borough", "Zone"])
    .assign(event_time=lambda d: d["event_time"].dt.tz_convert(None)))
borough_counts = bucket_counts_by_group(in_scope, freq="15min", group_cols="Borough")
zone_counts = bucket_counts_by_group(in_scope, freq="15min", group_cols="Zone")
print("Counts prepared", borough_counts.shape)

Counts prepared (2983, 6)


## 1. Poisson Diagnostics

In [12]:
zone = "Manhattan"
counts = borough_counts[zone]
poisson_summary(counts)

{'mean': np.float64(887.3442842775729),
 'variance': np.float64(292789.67798191315),
 'dispersion_index': np.float64(329.9617557353023),
 'chi2_stat': np.float64(127.21004535184139),
 'chi2_pvalue': np.float64(0.003225525284353077)}

## 2. NHPP and WLLN

In [13]:
nhpp = build_nhpp(counts, freq="15min")
emp_cum = cumulative_counts(counts)
exp_cum = nhpp.expected_counts(counts.index)
fig = go.Figure()
fig.add_trace(go.Scatter(x=emp_cum.index, y=emp_cum.values, name="Empirical"))
fig.add_trace(go.Scatter(x=exp_cum.index, y=exp_cum.values, name="NHPP E[N(t)]"))
fig.show()

### WLLN

In [14]:
def cumulative_average(series):
    n = np.arange(1, len(series)+1)
    return series.cumsum() / n

fig = go.Figure()
avg = cumulative_average(counts)
fig.add_trace(go.Scatter(x=avg.index, y=avg.values, name="Cumulative avg"))
fig.add_hline(y=counts.mean(), line_dash="dash")
fig.show()

In [15]:
rush_mask = counts.index.hour.isin([7,8,9,16,17,18,19])
quiet_mask = counts.index.hour.isin([1,2,3])
for label, mask, color in [("rush", rush_mask, "red"), ("quiet", quiet_mask, "green")]:
    avg = cumulative_average(counts[mask])
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=avg.index, y=avg.values, name=f"{label} avg", line=dict(color=color)))
    fig.add_hline(y=counts[mask].mean(), line_dash="dash")
    fig.update_layout(title=f"WLLN – {label}")
    fig.show()

## 3. Poisson vs Negative Binomial GLM

In [16]:
poisson_fit = fit_glm(counts, family="poisson")
nb_fit = fit_glm(counts, family="nb")
print("AIC poisson", poisson_fit.aic)
print("AIC nb", nb_fit.aic)
print("Dispersion poisson", poisson_fit.dispersion)
print("Dispersion nb", nb_fit.dispersion)

AIC poisson 365799.0546294417
AIC nb 45126.470432852206
Dispersion poisson 136.53981282037688
Dispersion nb 0.38915771830043905


/Users/atharvramesh/miniconda3/envs/nyc/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning:

Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.



In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=poisson_fit.residuals.index, y=poisson_fit.residuals.values, name="Poisson residuals"))
fig.add_trace(go.Scatter(x=nb_fit.residuals.index, y=nb_fit.residuals.values, name="NB residuals"))
fig.show()

## 4. CLT Approximation

In [18]:
daily_counts = counts.resample("1D").sum()
mu = daily_counts.mean()
sigma = daily_counts.std()
x = np.linspace(daily_counts.min(), daily_counts.max(), 200)
fig = go.Figure()
fig.add_histogram(x=daily_counts, nbinsx=30, histnorm="probability")
fig.add_trace(go.Scatter(x=x, y=norm.pdf(x, loc=mu, scale=sigma), name="Normal"))
fig.show()

## 5. Markov & Chebyshev Bounds

In [19]:
threshold = daily_counts.quantile(0.95)
emp_prob = (daily_counts >= threshold).mean()
markov = mu / threshold
chebyshev = (sigma**2) / ((threshold - mu)**2)
print(f"Empirical tail prob: {emp_prob:.4f}")
print(f"Markov bound: {markov:.4f}")
print(f"Chebyshev bound: {chebyshev:.4f}")

Empirical tail prob: 1.0000
Markov bound: inf
Chebyshev bound: 252.7353


/var/folders/hw/zpghgr8x4ybc9d8tnpvwmn780000gn/T/ipykernel_73140/41201936.py:3: RuntimeWarning:

divide by zero encountered in scalar divide



## 6. Chernoff Bound (Poisson approx)

In [20]:
lam = counts.mean() * 96
threshold = lam * 1.5
delta = threshold / lam - 1
chernoff = np.exp(-lam * ((1+delta)*np.log(1+delta) - delta))
print(f"Chernoff bound for 1.5x mean: {chernoff:.6e}")

Chernoff bound for 1.5x mean: 0.000000e+00


## 7. Zone-level GLM summary

In [21]:
top_zones = zone_counts.sum().sort_values(ascending=False).head(10).index
rows = []
for zn in top_zones:
    models = compare_models(zone_counts[zn])
    rows.append({
        "zone": zn,
        "poisson_aic": models["poisson"].aic,
        "poisson_disp": models["poisson"].dispersion,
        "nb_aic": models["nb"].aic,
        "nb_disp": models["nb"].dispersion,
    })
pd.DataFrame(rows)

/Users/atharvramesh/miniconda3/envs/nyc/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning:

Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.

/Users/atharvramesh/miniconda3/envs/nyc/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning:

Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.

/Users/atharvramesh/miniconda3/envs/nyc/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning:

Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.

/Users/atharvramesh/miniconda3/envs/nyc/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning:

Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.

/Users/atharvramesh/miniconda3/envs/nyc/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning:

Negative binomia

,zone,poisson_aic,poisson_disp,nb_aic,nb_disp
0,JFK Airport,32319.515250,5.709903,27784.527639,0.246778
1,Midtown Center,31210.535612,6.763755,25913.898122,0.480446
2,Upper East Side South,31767.910877,7.120134,25793.568227,0.541369
3,Upper East Side North,30414.350081,6.294494,24989.568316,0.587334
4,Midtown East,27024.502567,4.972092,25082.973538,0.431188
5,Times Sq/Theatre District,29291.180236,5.094453,25642.763896,0.224435
6,Penn Station/Madison Sq West,39232.427924,8.647321,26014.254542,0.383144
7,Lincoln Square East,36605.355855,10.505367,24553.599009,0.748305
8,LaGuardia Airport,29669.852888,6.337256,22812.605606,0.668079
9,Upper West Side South,27603.612240,6.705976,23941.956139,0.700706
